In [ ]:
#Use 3.12.4
import openai
import os
import copy
import time
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.docstore.document import Document

key_path = '/Users/jaesolshin/key/openai_key.txt'
os.environ["OPENAI_API_KEY"] = open(key_path, 'r', encoding='utf-8').read()
openai.api_key = os.environ["OPENAI_API_KEY"] 

In [ ]:
#!pip install pymupdf
from langchain.document_loaders import PyMuPDFLoader

path = '/Users/jaesolshin/Library/CloudStorage/GoogleDrive-ysys143@gmail.com/내 드라이브/2024-2/Google ML Bootcamp2024/data/bokbot/annual_report/2023년도 연차보고서.pdf'
loader = PyMuPDFLoader(path)
documents = loader.load()

In [ ]:
type(documents[0])

In [ ]:
print(f'문서개수: {len(documents)}')
print(f'첫번째 문서의 내용: {documents[0].page_content}')
print(f'첫번째 문서의 메타데이터: {documents[0].metadata}')

In [17]:
#!pip install spacy
#!spacy download ko_core_news_sm
from langchain.text_splitter import SpacyTextSplitter #자연어 처리를 위한 여러 기능을 제공한다.
text_splitter = SpacyTextSplitter(
    chunk_size=2000, #분할할 크기를 설정
    chunk_overlap=10,
    pipeline='ko_core_news_sm' #분할에 사용할 언어모델을 설정
)

splitted_documents = text_splitter.split_documents(documents) #문서를 분할

print(f'분할 전 문서 개수: {len(documents)}')
print(f'분할 후 문서 개수: {len(splitted_documents)}')

분할 전 문서 개수: 191
분할 후 문서 개수: 180


In [18]:
splitted_documents[2].page_content

'총재 서문\n한국은행은우리나라중앙은행으로서통화신용정책의효율적수행을통해\n물가안정과금융안정을도모함으로써국민경제의건전한발전에이바지하고\n자최선을다하고있습니다.\n\n\n2023년은급변하는안팎의여건에대응하여보다나은정책과해법을마련\n하는데분주했던한해였습니다.\n\n인플레이션을잡기위한주요국중앙은행들\n의가파른금리인상에이어미국실리콘밸리은행사태, 이스라엘-하마스전쟁\n등으로경계의끈을한시도늦출수없는긴장된순간의연속이었습니다.\n\n이\n처럼높아진대외여건의불확실성속에서한국은행은최우선으로물가안정을\n도모하는가운데금융안정측면의리스크를면밀히점검하며통화정책을운영\n하였습니다.\n\n기준금리를3.5%의긴축적인수준으로유지하고, 새마을금고예금\n인출사태등금융･외환시장불안에는시장안정화조치를통해적극대처하\n였습니다.\n\n\n한국은행은경제상황에대한정확한진단과정교한정책대응을뒷받침하고자\n조사및정책연구업무를적극적으로수행하였습니다.\n\n초저출산및초고령사회\n의극단적인구구조문제, 장기구조적관점에서진단한가계부채현황, 지역간\n인구이동과지역경제등에대한보고서를통해우리나라가해결해야할중장기\n과제를제시했습니다.\n\n또한기후변화및AI 확산등최신현안에대해서도다'

In [19]:
print(splitted_documents[4].page_content)

금융통화위원회
의 장  이 창 용
위 원  조 윤 제
위 원  서 영 경  
위 원  신 성 환      
위 원  장 용 성   
위 원  유 상 대
위 원  황 건 일


In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

# 모델 정의
chat = ChatOpenAI(
    model='gpt-4o-mini-2024-07-18',
)

# 1단계 프롬프트(띄어쓰기)
examples = [
    {
        'input':"총재 서문\n한국은행은우리나라중앙은행으로서통화신용정책의효율적수행을통해\n물가안정과금융안정을도모함으로써국민경제의건전한발전에이바지하고\n자최선을다하고있습니다.\n\n\n2023년은급변하는안팎의여건에대응하여보다나은정책과해법을마련\n하는데분주했던한해였습니다.\n\n인플레이션을잡기위한주요국중앙은행들\n의가파른금리인상에이어미국실리콘밸리은행사태, 이스라엘-하마스전쟁\n등으로경계의끈을한시도늦출수없는긴장된순간의연속이었습니다.\n\n이\n처럼높아진대외여건의불확실성속에서한국은행은최우선으로물가안정을\n도모하는가운데금융안정측면의리스크를면밀히점검하며통화정책을운영\n하였습니다.",
        'output':"총재 서문 한국은행은 우리나라 중앙은행으로서 통화신용정책의 효율적 수행을 통해 물가안정과 금융안정을 도모함으로써 국민경제의 건전한 발전에 이바지하고자 최선을 다하고 있습니다. 2023년은 급변하는 안팎의 여건에 대응하여 보다 나은 정책과 해법을 마련하는데 분주했던 한 해였습니다. 인플레이션을 잡기 위한 주요국 중앙은행들의 가파른 금리인상에 이어 미국 실리콘밸리 은행사태, 이스라엘-하마스전쟁 등으로 경계의 끈을 한시도 늦출 수 없는 긴장된 순간의 연속이었습니다. 이처럼 높아진 대외여건의 불확실성 속에서 한국은행은 최우선으로 물가안정을 도모하는 가운데 금융안정측면의 리스크를 면밀히 점검하며 통화정책을 운영하였습니다."
    },
    {
        'input': "기준금리를3.5%의긴축적인수준으로유지하고, 새마을금고예금\n인출사태등금융･외환시장불안에는시장안정화조치를통해적극대처하\n였습니다.\n\n\n한국은행은경제상황에대한정확한진단과정교한정책대응을뒷받침하고자\n조사및정책연구업무를적극적으로수행하였습니다.\n\n초저출산및초고령사회\n의극단적인구구조문제, 장기구조적관점에서진단한가계부채현황, 지역간\n인구이동과지역경제등에대한보고서를통해우리나라가해결해야할중장기\n과제를제시했습니다.\n\n또한기후변화및AI 확산등최신현안에대해서도다",
        'output': "기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융･외환시장 불안에는 시장안정화조치를 통해 적극대처하였습니다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였습니다. 초저출산 및 초고령사회의 극단적 인구구조문제, 장기구조적관점에서 진단한 가계부채현황, 지역간 인구이동과 지역경제 등에 대한 보고서를통해 우리나라가 해결해야 할 중장기 과제를제시했습니다. 또한 기후변화 및 AI 확산 등 최신현안에 대해서도 다"
    },

]

prompt = PromptTemplate(
    input_variables=['input', 'output'],
    template='입력: {input}\n출력: {output}'
)

few_shot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=prompt,
    prefix="예시와 같이 띄어쓰기가 되어있지 않은 문장을 올바른 형식으로 수정하세요. 1) 적절한 띄어쓰기가 이루어져야 합니다. 2) 문장에 포함된 줄바꿈 '\n'은 제거하세요. 3) input 문장의 내용을 임의로 삭제하지 마세요. 문장이 중간에 끊기더라 끝까지 변환하세요. 5) input 문장에 들어있지 않은 내용을 임의로 추가하지 마세요.",
    suffix="입력: {input_string}\n출력:",
    input_variables=['input_string'],
)

formatted_prompt = few_shot_prompt.format(
    input_string=splitted_documents[5].page_content
)


# test
result = chat(
    [
        SystemMessage(content='당신은 신문사에서 전문적인 교열을 담당하는 편집자입니다. 요구사항에 맞게 주어진 텍스트를 수정하세요.'),
        HumanMessage(content=formatted_prompt)
    ]
)

print('result:', result.content)

result: 차례 Ⅰ. 경제 동향 1 1. 실물 경제 3 2. 금융･외환 시장 15 Ⅱ. 업무 현황 25 1. 통화 신용 정책 28 2. 금융 안정 36 3. 발권 45 4. 지급 결제 51 5. 외환 및 국제 협력 58 6. 조사 통계 69 7. 국고 및 증권 81 8. 대외 커뮤니케이션 85 Ⅲ. 경영 상황 95 1. 조직 운영 97 2. 재무 현황 108 부록 123 1. 한국은행 개요 124 2. 금융통화위원회의 주요 의결 및 논의 사항 133 3. 경제 일지 163 4. 부문별 담당 부서 및 집필자 170 참고 Ⅱ-1. 녹색 금융 국제 컨퍼런스 개최 44 Ⅱ-2. 자동화 금고 시스템 가동 50 Ⅱ-3. 한국은행 총재, 국제결제은행(BIS) ｢글로벌 금융 시스템위원회(CGFS)｣ 의장 선임 68 Ⅱ-4. 한국은행 금융･경제 스냅샷 서비스 개시 80 Ⅱ-5. 新전략체계 가동 93 Ⅲ-1. 체험형 청년 인턴 프로그램 도입 106 Ⅲ-2. 한국은행의 다양성･포용성 현황 점검 107


In [21]:
rectified_documents_step1=[]

def rectify(phrase):
    formatted_prompt = few_shot_prompt.format(
        input_string=phrase
    )
    result = chat(
        [
            SystemMessage(content='당신은 신문사에서 전문적인 교열을 담당하는 편집자입니다. 요구사항에 맞게 주어진 텍스트를 수정하세요.'),
            HumanMessage(content=formatted_prompt)
        ]
    )
    
    return result.content

for idx, doc in enumerate(splitted_documents[30:40]):
    print(f'processing {idx+1}')
    rect = rectify(doc.page_content)
    rectified_doc = Document(page_content=rect, metadata=doc.metadata)
    rectified_documents_step1.append(rectified_doc)
    time.sleep(1)

processing 1
processing 2
processing 3
processing 4
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10


In [22]:
for doc in rectified_documents_step1:
    print(doc.page_content)

기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융･외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였습니다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였습니다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시했습니다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다관은 일부 업권의 건전성 우려, 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었습니다. 자산운용사 수신은 주가 상승, 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었습니다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었습니다. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설‧부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭 축소되었습니다. 한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었습니다. 가계대출은 주택 매매 거래 증가 및 대출 규제 완화, 주택금융공사의 특례 보금자리론 등 정책 금융상품 공급 확대으로 예금은행 주택담보대출과 기타 금융기관 대출이 늘어나면서 증가 전환하였습니다. 다만, 높은 금리 수준 및 총부채 원리금 상환 비율(DSR, Debt Service Ratio) 규제, 지방 부동산 경기 회복 지연 등의 영향으로 예금은행 기타 대출과 비은행 예금 취급 기관은 감소세를 지속하였습니다.
21 Ⅰ 경제 동향 2 금융･외환시장 (4) 외환시장 원/달러 환율 상승, 변동성 소폭 축소 2023년 말 원/달러 환율(1,288.0원)

In [28]:
# 2단계 프롬프트(표, 목차 제거)
examples2 = [
    {
        'input':"Ⅱ- 1. 주요 이슈 44 Ⅱ- 2.",
        'output':"\n"
    },
    {
        'input':"표 Ⅱ- 13. 외화자산 구성 내역 (기말 기준) (%, %p) 2021 2022(A) 2023(B) 증감(B-A) ",
        'output':"\n"
    },
    {
        'input':"127) 한국은행의 업무 관련 지식, 정책 수행 경험...",
        'output':"\n"
    },
    {
        'input':"주요 품목별 수출 (%)\n 2022 2023\n 증감률 비중 증감률 비중\n IT제품 2.5 <34.1> -19.9 <29.5> ...",
        'output':"\n"
    },
]

few_shot_prompt = FewShotPromptTemplate(
    examples=examples2,
    example_prompt=prompt,
    prefix="예시와 같이 표나 차례, 주석 형식의 불필요한 텍스트를 제거하세요. 1) 한국어 문법에 맞는 문장이 아닌 기호나 숫자, 단어 등이 나열된 목차 형식의 텍스트 제거한다. 2) 기호나 숫자, 단어 등이 나열된 표 형식의 데이터 제거한다. 3) 주석은 '127)'과 같이 숫자로 시작하는 문장은 주석이므로 제거한다. 4)  문체는 '입니다'가 아닌 '이다'체로 통일한다.",
    suffix="입력: {input_string}\n출력:",
    input_variables=['input_string'],
)

rectified_documents_step2 =[]

def rectify2(phrase):
    formatted_prompt = few_shot_prompt.format(
        input_string=phrase
    )
    result = chat(
        [
            SystemMessage(content='당신은 신문사에서 전문적인 교열을 담당하는 편집자입니다. 요구사항에 맞게 주어진 텍스트를 수정하세요.'),
            HumanMessage(content=formatted_prompt)
        ]
    )
    
    return result.content

for idx, doc in enumerate(rectified_documents_step1[:10]):
    print(f'processing {idx+1}')
    rect = rectify2(doc.page_content)
    
    # '입력:'이나 '출력:'이 포함되어 있는지 확인
    if '입력:' not in rect and '출력:' not in rect:
        rectified_doc = Document(page_content=rect, metadata=doc.metadata)
        rectified_documents_step2.append(rectified_doc)
    else:
        print(f'Skipping document {idx+1} due to "입력:" or "출력:" being present.')
    
    time.sleep(1)


processing 1
processing 2
processing 3
processing 4
Skipping document 4 due to "입력:" or "출력:" being present.
processing 5
processing 6
processing 7
processing 8
processing 9
processing 10


In [29]:
for doc in rectified_documents_step2:
    print(doc.page_content)

기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융 및 외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역 간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각도로 분석하였다. 

일부 업권의 건전성 우려와 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승과 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설 및 부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭으로 축소되었다. 

한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었다. 가계대출은 주택 매매 거래 증가 및 대출 규제 완화, 주택금융공사의 특례 보금자리론 등 정책 금융상품 공급 확대에 따라 예금은행 주택담보대출과 기타 금융기관 대출이 늘어나면서 증가 전환하였다. 다만, 높은 금리 수준과 총부채 원리금 상환 비율(DSR) 규제, 지방 부동산 경기 회복 지연 등의 영향으로 예금은행 기타 대출과 비은행 예금 취급 기관은 감소세를 지속하였다.
경제 동향 금융·외환시장 외환시장 원/달러 환율 상승, 변동성 소폭 축소 2023년 말 원/달러 환율(1,288.0원)은 2022년 말(1,264.5원) 대비 23.5원 상

In [35]:
txt = ""

for doc in rectified_documents_step2:
    txt += doc.page_content

#txt = txt.replace("\n", " ")  # "\n" 제거

In [36]:
print(txt)

기준금리를 3.5%의 긴축적인 수준으로 유지하고, 새마을금고 예금인출사태 등 금융 및 외환시장 불안에는 시장안정화 조치를 통해 적극 대처하였다. 한국은행은 경제상황에 대한 정확한 진단과 정교한 정책대응을 뒷받침하고자 조사 및 정책연구업무를 적극적으로 수행하였다. 초저출산 및 초고령사회의 극단적인 인구구조 문제, 장기구조적 관점에서 진단한 가계부채 현황, 지역 간 인구이동과 지역경제 등에 대한 보고서를 통해 우리나라가 해결해야 할 중장기 과제를 제시하였다. 또한 기후변화 및 AI 확산 등 최신 현안에 대해서도 다각도로 분석하였다. 

일부 업권의 건전성 우려와 금리 메리트 축소 등으로 감소하거나 증가 폭이 둔화되었다. 자산운용사 수신은 주가 상승과 대내외 긴축기조 완화 기대 등으로 주식형 및 채권형 펀드로 자금이 유입되면서 증가 폭이 확대되었다. 2023년 중 기업대출은 은행과 비은행 예금 취급 기관 모두 전년 대비 증가 폭이 축소되었다. 특히 비은행 예금 취급 기관 기업대출은 부동산 경기 부진 등으로 건설 및 부동산업 관련 대출 수요가 크게 둔화되면서 증가 규모가 전년 대비 큰 폭으로 축소되었다. 

한편, 은행 기업대출의 경우 대기업 대출은 기업들이 회사채 대신 대출을 통한 자금 조달을 늘리면서 전년에 이어 높은 증가세를 보였으나, 중소기업 대출은 개인사업자에 대한 코로나19 지원 대출의 상환이 늘면서 증가 폭이 축소되었다. 가계대출은 주택 매매 거래 증가 및 대출 규제 완화, 주택금융공사의 특례 보금자리론 등 정책 금융상품 공급 확대에 따라 예금은행 주택담보대출과 기타 금융기관 대출이 늘어나면서 증가 전환하였다. 다만, 높은 금리 수준과 총부채 원리금 상환 비율(DSR) 규제, 지방 부동산 경기 회복 지연 등의 영향으로 예금은행 기타 대출과 비은행 예금 취급 기관은 감소세를 지속하였다.경제 동향 금융·외환시장 외환시장 원/달러 환율 상승, 변동성 소폭 축소 2023년 말 원/달러 환율(1,288.0원)은 2022년 말(1,264.5원) 대비 23.5원 상승

In [27]:
open('rectified_text_2023_2.txt', 'w', encoding='utf-8').write(txt)

8654